In [1]:
import json
import numpy as np
from sklearn.metrics import classification_report
import pickle

In [2]:
special_characters = ['.',
                     ',',
                     ':',
                     '~',
                     '/',
                     '?',
                     '!',
                     '`',
                     '@',
                     '#',
                     '$',
                     '%',
                     '^',
                     '&',
                     '*',
                     '(',
                     ')',
                     '-',
                     '_',
                     '=',
                     '+',
                     '{',
                     '}',
                     '[',
                     ']',
                     '\\',
                     '|',
                     ';',
                     '\'',
                     '"',
                     '<',
                     '>'
                     ]

In [13]:
train_path = 'Data/traindata.json'
test_path = 'Data/testdata.json'

with open(train_path,'r',encoding='utf8') as train_file:
    train_data_raw = train_file.readlines()
    
with open(test_path,'r',encoding='utf8') as test_file:
    test_data_raw = test_file.readlines()
    
train_data_dict = {}
for i in range(len(train_data_raw)):
    train_data_dict[i] = json.loads(train_data_raw[i])
    
test_data_dict = {}
for i in range(len(test_data_raw)):
    test_data_dict[i] = json.loads(test_data_raw[i])

In [12]:

    
    
entity_to_intertag = {
    'Name' : '1010101', #
    'College Name' : '1111111', #
    'Degree' : '2222222',
    'Graduation Year' : '3333333',
    'Years of Experience' : '4444444',
    'Companies worked at' : '5555555',
    'Designation' : '6666666', # 
    'Skills' : '7777777', #
    'Location' : '8888888', #
    'Email Address' : '9999999' #
}


intertag_to_label = {
    '1010101' : '10',
    '1111111' : '1',
    '2222222' : '2',
    '3333333' : '3',
    '4444444' : '4',
    '5555555' : '5',
    '6666666' : '6',
    '7777777' : '7',
    '8888888' : '8',
    '9999999' : '9'
}


train_y = []
train_x_raw = []


train_y = []
train_x_raw = []
    
for i in range(len(train_data_dict)):
    #print(i)
    #i = 174
    text = train_data_dict[i]['content']
    for spec in special_characters:
        text = text.replace(spec,' '+spec+' ')
    train_x_raw.append(text)
    #print(len(train_data_dict[i]['content'].split()))
    for index, label in enumerate(train_data_dict[i]['annotation']):
       
        start_index = label['points'][0]['start']
        end_index = label['points'][0]['end']
        
        if len(label['label']) != 0:
            if label['label'][0] in list(entity_to_intertag.keys()):
                tag = entity_to_intertag[label['label'][0]]
                
                #print(start_index,end_index)
                text = train_data_dict[i]['content'][start_index:end_index+1]
                #print(text)
                for spec in special_characters:
                    text = text.replace(spec,' '+spec+' ')
                #print(text)
                
                to_be_replaced = ' '.join([tag] * len(text.split()))
                #print(to_be_replaced)
                
                if train_data_dict[i]['content'][start_index:end_index+1][0]==' ' and train_data_dict[i]['content'][start_index:end_index+1][-1]==' ':
                    train_data_dict[i]['content'] = train_data_dict[i]['content'][:start_index]+' ' + to_be_replaced +' '+ train_data_dict[i]['content'][end_index + 1:]
                elif train_data_dict[i]['content'][start_index:end_index+1][0]==' ':
                    train_data_dict[i]['content'] = train_data_dict[i]['content'][:start_index]+' '+ to_be_replaced + train_data_dict[i]['content'][end_index + 1:]
                elif train_data_dict[i]['content'][start_index:end_index+1][-1]==' ':
                    train_data_dict[i]['content'] = train_data_dict[i]['content'][:start_index]+ to_be_replaced +' '+ train_data_dict[i]['content'][end_index + 1:]
                elif train_data_dict[i]['content'][start_index:end_index+1][0]=='\n' and train_data_dict[i]['content'][start_index:end_index+1][-1]=='\n':
                    train_data_dict[i]['content'] = train_data_dict[i]['content'][:start_index]+'\n' + to_be_replaced +'\n'+ train_data_dict[i]['content'][end_index + 1:]
                elif train_data_dict[i]['content'][start_index:end_index+1][0]=='\n':
                    train_data_dict[i]['content'] = train_data_dict[i]['content'][:start_index]+'\n'+ to_be_replaced + train_data_dict[i]['content'][end_index + 1:]
                elif train_data_dict[i]['content'][start_index:end_index+1][-1]=='\n':
                    train_data_dict[i]['content'] = train_data_dict[i]['content'][:start_index]+ to_be_replaced +'\n'+ train_data_dict[i]['content'][end_index + 1:]    
                else:
                    train_data_dict[i]['content'] = train_data_dict[i]['content'][:start_index]+ to_be_replaced + train_data_dict[i]['content'][end_index + 1:]
                z = []
                
                #print(train_data_dict[i]['content'])
                
                
                for token in train_data_dict[i]['content'].split():
                    for tag in list(entity_to_intertag.values()):
        
                        if tag in token:
                            token = tag
                    z.append(token) 
                
    text = ' '.join(train_data_dict[i]['content'].split())
    for spec in special_characters:
        text = text.replace(spec,' '+spec+' ')
    
    z = text.split()
    
                #print(len(train_data_dict[i]['content'].split()))
    #train_data_dict[i]['content'] = [tag if tag in token else token for tag in list(entity_to_intertag.values()) for token in train_data_dict[i]['content'].split()]
    
    temp_label = ['0' if token not in list(entity_to_intertag.values()) else token for token in z]
    train_y.append([int(intertag_to_label[label]) if label in list(intertag_to_label.keys()) else int(label) for label in temp_label])
    #print(len(temp_label))
    
    
test_y = []
test_x_raw = []


for i in range(len(test_data_dict)):
    
    text = test_data_dict[i]['content']
    for spec in special_characters:
        text = text.replace(spec,' '+spec+' ')
    test_x_raw.append(text)
    
    
    #print(i)
    #print(len(test_data_dict[i]['content'].split()))
    for index, label in enumerate(test_data_dict[i]['annotation']):
       
        start_index = label['points'][0]['start']
        end_index = label['points'][0]['end']
        
        if len(label['label']) != 0:
            if label['label'][0] in list(entity_to_intertag.keys()):
                tag = entity_to_intertag[label['label'][0]]
                
                
                text = test_data_dict[i]['content'][start_index:end_index+1]
                for spec in special_characters:
                    text = text.replace(spec,' '+spec+' ')
                to_be_replaced = ' '.join([tag] * len(text.split()))
                
                
                if test_data_dict[i]['content'][start_index:end_index+1][0]==' ' and test_data_dict[i]['content'][start_index:end_index+1][-1]==' ':
                    test_data_dict[i]['content'] = test_data_dict[i]['content'][:start_index]+' ' + to_be_replaced +' '+ test_data_dict[i]['content'][end_index + 1:]
                elif test_data_dict[i]['content'][start_index:end_index+1][0]==' ':
                    test_data_dict[i]['content'] = test_data_dict[i]['content'][:start_index]+' '+ to_be_replaced + test_data_dict[i]['content'][end_index + 1:]
                elif test_data_dict[i]['content'][start_index:end_index+1][-1]==' ':
                    test_data_dict[i]['content'] = test_data_dict[i]['content'][:start_index]+ to_be_replaced +' '+ test_data_dict[i]['content'][end_index + 1:]
                elif test_data_dict[i]['content'][start_index:end_index+1][0]=='\n' and test_data_dict[i]['content'][start_index:end_index+1][-1]=='\n':
                    test_data_dict[i]['content'] = test_data_dict[i]['content'][:start_index]+'\n' + to_be_replaced +'\n'+ test_data_dict[i]['content'][end_index + 1:]
                elif test_data_dict[i]['content'][start_index:end_index+1][0]=='\n':
                    test_data_dict[i]['content'] = test_data_dict[i]['content'][:start_index]+'\n'+ to_be_replaced + test_data_dict[i]['content'][end_index + 1:]
                elif test_data_dict[i]['content'][start_index:end_index+1][-1]=='\n':
                    test_data_dict[i]['content'] = test_data_dict[i]['content'][:start_index]+ to_be_replaced +'\n'+ test_data_dict[i]['content'][end_index + 1:]    
                else:
                    test_data_dict[i]['content'] = test_data_dict[i]['content'][:start_index]+ to_be_replaced + test_data_dict[i]['content'][end_index + 1:]
                z = []
                for token in test_data_dict[i]['content'].split():
                    for tag in list(entity_to_intertag.values()):
        
                        if tag in token:
                            token = tag
                    z.append(token) 
    
    text = ' '.join(test_data_dict[i]['content'].split())
    for spec in special_characters:
        text = text.replace(spec,' '+spec+' ')
    
    z = text.split()
    temp_label = ['0' if token not in list(entity_to_intertag.values()) else token for token in z]
    test_y.append([int(intertag_to_label[label]) if label in list(intertag_to_label.keys()) else int(label) for label in temp_label])
    #print(len(temp_label))
    


In [4]:
## Elmo embedding
'''
import tensorflow_hub as hub
import tensorflow as tf
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

def elmo_vectors(x):
    embeddings = elmo(x, signature="default", as_dict=True)["elmo"]
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        # return average of ELMo features
        return sess.run(embeddings)
count = 0
train_x = []
for x in train_x_raw:
    print('\n')
    count += 1
    print(count)
    print(len(x.split()))
    elmo_x = elmo_vectors(x.split())
    print(elmo_x.shape)
    train_x.append(elmo_x)
    
count = 0
test_x = []
for x in test_x_raw:
    count += 1
    print(count)
    test_x.append(elmo_vectors(x.split()))

    

'''
#with open('train_x_c.pickle','wb') as tfile:
#    pickle.dump(train_x,tfile)
#with open('test_c.pickle','wb') as tfile:
#    pickle.dump(test_x,tfile)    


'\nimport tensorflow_hub as hub\nimport tensorflow as tf\nelmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)\n\ndef elmo_vectors(x):\n    embeddings = elmo(x, signature="default", as_dict=True)["elmo"]\n    with tf.Session() as sess:\n        sess.run(tf.global_variables_initializer())\n        sess.run(tf.tables_initializer())\n        # return average of ELMo features\n        return sess.run(embeddings)\ncount = 0\ntrain_x = []\nfor x in train_x_raw:\n    print(\'\n\')\n    count += 1\n    print(count)\n    print(len(x.split()))\n    elmo_x = elmo_vectors(x.split())\n    print(elmo_x.shape)\n    train_x.append(elmo_x)\n    \ncount = 0\ntest_x = []\nfor x in test_x_raw:\n    count += 1\n    print(count)\n    test_x.append(elmo_vectors(x.split()))\n\n    \n\n'

In [5]:

with open('train_x_c.pickle','rb') as tfile:
    train_x = pickle.load(tfile)
    
with open('test_c.pickle','rb') as tfile:
    test_x = pickle.load(tfile)
    
    
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)


def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
device = get_default_device()

train_x_list = [torch.from_numpy(x).to(device) for x in train_x]
test_x_list = [torch.from_numpy(x).to(device) for x in test_x]

train_y_list = [torch.tensor(y,dtype=np.long).to(device) for y in train_y]
test_y_list = [torch.tensor(y,dtype=np.long).to(device) for y in test_y]

train_data_full = list(zip(train_x_list,train_y_list))
test_data_full = list(zip(test_x_list,test_y_list))

In [6]:
# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 1024
HIDDEN_DIM = 256
MODEL_PATH = r'sequence_labelling_model_bidirectional.pth'


LAST_EPOCH_MODEL = r'sequence_labelling_model_bidirectional_last.pth'

In [7]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        #self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True,dropout=0.25)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim*2, tagset_size)

    def forward(self, sentence):
        #embeds = self.word_embeddings(sentence)
        #print(sentence.shape)
        #print(embeds.view(len(sentence), 1, -1).shape)
        #lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        lstm_out, _ = self.lstm(sentence)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [8]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, 11)
loss_function = nn.NLLLoss()

#loss_function = nn.BCEWithLogitsLoss(reduction=None)


optimizer = optim.SGD(model.parameters(), lr=0.1)
model.to(device)


previous_best_validation = None
for epoch in range(100):  # again, normally you would NOT do 300 epochs, it is toy data
    print(epoch+1)
    count = 0
    running_loss = 0.0
    running_loss_Val = 0.0
    for sentence, tags in train_data_full[:180]:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()
        #print(count)
        count += 1
        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Step 3. Run our forward pass.
        tag_scores = model(sentence)
        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        
        loss = loss_function(tag_scores, tags)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if count % 20 == 19:    # print every 2000 mini-batches♠
            print('[%d, %5d] training loss: %.3f' %
                  (epoch + 1, count + 1, running_loss / 20))
            running_loss = 0.0
    predicted_val = []
    actual_val = []
    with torch.no_grad():
        count_val = 0
        for sentence_val, tags_val in train_data_full[180:]:
            #print(count_val)
            count_val += 1
            tag_scores_Val = model(sentence_val)
            loss_Val = loss_function(tag_scores_Val, tags_val)
            running_loss_Val += loss_Val.item()
            predicted_val.extend(torch.argmax(tag_scores_Val,1).to('cpu').detach().numpy().tolist())
            actual_val.extend(tags_val.to('cpu').detach().numpy().tolist())
            if count_val % 20 == 19:    # print every 20 mini-batches♠
                validation_loss = running_loss_Val / 20
                print('[%d, %5d] validation loss: %.3f' %
                          (epoch + 1, count_val+1, validation_loss))
                running_loss_Val = 0.0
        
        '''if previous_best_validation is None:
            previous_best_validation = validation_loss + 1

        if validation_loss < previous_best_validation:
            print('validation loss decreased')
            print('saving best model: %.3f' % (validation_loss))
            torch.save(model, MODEL_PATH)
            previous_best_validation = validation_loss'''
        
            
        clr = classification_report(actual_val, predicted_val, target_names=[ 'Others',            # 0
                                                            'College Name',        # 1
                                                            'Degree',              # 2
                                                            'Graduation Year',     # 3
                                                            'Years of Experience', # 4
                                                            'Companies worked at', # 5
                                                            'Designation',         # 6
                                                            'Skills',              # 7
                                                            'Location',            # 8
                                                            'Email Address',       # 9 
                                                            'Name'])              # 10
        #f1 = 0
        #for i in clr.keys():

         #   if i not in ['Others','accuracy','macro avg','weighted avg']:
         #       print(i)
          #      print(np.round(clr.get(i)['f1-score'],2))
           #     f1 += clr.get(i)['f1-score']
        print(clr)        

c:\programdata\anaconda3\envs\sequence_labelling\lib\site-packages\torch\nn\modules\rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


1
[1,    20] training loss: 0.990
[1,    40] training loss: 0.780
[1,    60] training loss: 0.676
[1,    80] training loss: 0.766
[1,   100] training loss: 0.693
[1,   120] training loss: 0.654
[1,   140] training loss: 0.720
[1,   160] training loss: 0.496
[1,   180] training loss: 0.612
[1,    20] validation loss: 0.529
                     precision    recall  f1-score   support

             Others       0.91      1.00      0.95     10158
       College Name       0.00      0.00      0.00       100
             Degree       0.00      0.00      0.00       140
    Graduation Year       0.00      0.00      0.00         8
Years of Experience       0.00      0.00      0.00         2
Companies worked at       0.00      0.00      0.00        64
        Designation       0.00      0.00      0.00       167
             Skills       0.86      0.43      0.58       414
           Location       0.00      0.00      0.00        40
      Email Address       0.38      0.01      0.02       235
    

c:\programdata\anaconda3\envs\sequence_labelling\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[2,    20] training loss: 0.538
[2,    40] training loss: 0.569
[2,    60] training loss: 0.456
[2,    80] training loss: 0.687
[2,   100] training loss: 0.541
[2,   120] training loss: 0.487
[2,   140] training loss: 0.548
[2,   160] training loss: 0.393
[2,   180] training loss: 0.492
[2,    20] validation loss: 0.451
                     precision    recall  f1-score   support

             Others       0.92      0.99      0.95     10158
       College Name       0.92      0.12      0.21       100
             Degree       1.00      0.02      0.04       140
    Graduation Year       0.00      0.00      0.00         8
Years of Experience       0.00      0.00      0.00         2
Companies worked at       0.00      0.00      0.00        64
        Designation       0.50      0.02      0.03       167
             Skills       0.72      0.64      0.67       414
           Location       0.00      0.00      0.00        40
      Email Address       0.82      0.12      0.21       235
      

[9,    20] training loss: 0.230
[9,    40] training loss: 0.328
[9,    60] training loss: 0.232
[9,    80] training loss: 0.411
[9,   100] training loss: 0.238
[9,   120] training loss: 0.224
[9,   140] training loss: 0.271
[9,   160] training loss: 0.271
[9,   180] training loss: 0.209
[9,    20] validation loss: 0.347
                     precision    recall  f1-score   support

             Others       0.96      0.96      0.96     10158
       College Name       0.78      0.52      0.62       100
             Degree       0.95      0.63      0.76       140
    Graduation Year       0.00      0.00      0.00         8
Years of Experience       0.00      0.00      0.00         2
Companies worked at       0.24      0.14      0.18        64
        Designation       0.82      0.29      0.43       167
             Skills       0.49      0.74      0.59       414
           Location       0.75      0.15      0.25        40
      Email Address       0.75      0.71      0.73       235
      

...

[72,    20] training loss: 0.034
[72,    40] training loss: 0.038
[72,    60] training loss: 0.055
[72,    80] training loss: 0.038
[72,   100] training loss: 0.050
[72,   120] training loss: 0.023
[72,   140] training loss: 0.024
[72,   160] training loss: 0.032
[72,   180] training loss: 0.023
[72,    20] validation loss: 0.512
                     precision    recall  f1-score   support

             Others       0.96      0.97      0.96     10158
       College Name       0.70      0.45      0.55       100
             Degree       0.99      0.56      0.71       140
    Graduation Year       0.17      0.12      0.14         8
Years of Experience       0.00      0.00      0.00         2
Companies worked at       0.16      0.12      0.14        64
        Designation       0.73      0.29      0.42       167
             Skills       0.61      0.78      0.69       414
           Location       0.59      0.40      0.48        40
      Email Address       0.74      0.84      0.79       

[79,    20] training loss: 0.026
[79,    40] training loss: 0.031
[79,    60] training loss: 0.043
[79,    80] training loss: 0.032
[79,   100] training loss: 0.048
[79,   120] training loss: 0.022
[79,   140] training loss: 0.021
[79,   160] training loss: 0.027
[79,   180] training loss: 0.020
[79,    20] validation loss: 0.524
                     precision    recall  f1-score   support

             Others       0.96      0.96      0.96     10158
       College Name       0.71      0.44      0.54       100
             Degree       0.96      0.59      0.73       140
    Graduation Year       0.17      0.12      0.14         8
Years of Experience       0.00      0.00      0.00         2
Companies worked at       0.18      0.16      0.17        64
        Designation       0.73      0.28      0.41       167
             Skills       0.59      0.79      0.67       414
           Location       0.57      0.40      0.47        40
      Email Address       0.73      0.84      0.78       


86
[86,    20] training loss: 0.021
[86,    40] training loss: 0.025
[86,    60] training loss: 0.039
[86,    80] training loss: 0.030
[86,   100] training loss: 0.041
[86,   120] training loss: 0.020
[86,   140] training loss: 0.016
[86,   160] training loss: 0.020
[86,   180] training loss: 0.015
[86,    20] validation loss: 0.578
                     precision    recall  f1-score   support

             Others       0.96      0.96      0.96     10158
       College Name       0.72      0.44      0.55       100
             Degree       0.99      0.59      0.74       140
    Graduation Year       0.17      0.12      0.14         8
Years of Experience       0.00      0.00      0.00         2
Companies worked at       0.12      0.11      0.12        64
        Designation       0.72      0.28      0.40       167
             Skills       0.57      0.78      0.66       414
           Location       0.55      0.40      0.46        40
      Email Address       0.73      0.86      0.79   

[93,    20] training loss: 0.015
[93,    40] training loss: 0.020
[93,    60] training loss: 0.034
[93,    80] training loss: 0.025
[93,   100] training loss: 0.035
[93,   120] training loss: 0.019
[93,   140] training loss: 0.012
[93,   160] training loss: 0.016
[93,   180] training loss: 0.013
[93,    20] validation loss: 0.608
                     precision    recall  f1-score   support

             Others       0.96      0.96      0.96     10158
       College Name       0.73      0.45      0.56       100
             Degree       0.96      0.56      0.71       140
    Graduation Year       0.25      0.12      0.17         8
Years of Experience       0.00      0.00      0.00         2
Companies worked at       0.14      0.11      0.12        64
        Designation       0.73      0.27      0.39       167
             Skills       0.58      0.78      0.66       414
           Location       0.55      0.40      0.46        40
      Email Address       0.74      0.87      0.80       

[100,    20] training loss: 0.160
[100,    40] training loss: 0.192
[100,    60] training loss: 0.143
[100,    80] training loss: 0.219
[100,   100] training loss: 0.102
[100,   120] training loss: 0.133
[100,   140] training loss: 0.094
[100,   160] training loss: 0.150
[100,   180] training loss: 0.074
[100,    20] validation loss: 0.393
                     precision    recall  f1-score   support

             Others       0.96      0.96      0.96     10158
       College Name       0.68      0.52      0.59       100
             Degree       0.95      0.67      0.79       140
    Graduation Year       0.29      0.25      0.27         8
Years of Experience       0.00      0.00      0.00         2
Companies worked at       0.20      0.17      0.19        64
        Designation       0.81      0.35      0.49       167
             Skills       0.54      0.76      0.63       414
           Location       0.52      0.42      0.47        40
      Email Address       0.69      0.85      0

In [ ]:
torch.save(model, LAST_EPOCH_MODEL)

In [ ]:
inference_model = torch.load(MODEL_PATH)
inference_model.eval()

In [ ]:
# See what the scores are after training
predicted = []
p = []
actual = []
a = []
with torch.no_grad():
    for sentence_val, tags_val in test_data_full:
        tag_scores = model(sentence_val)
        predicted.extend(torch.argmax(tag_scores,1).to('cpu').detach().numpy().tolist())
        actual.extend(tags_val.to('cpu').detach().numpy().tolist())
        p.append(torch.argmax(tag_scores,1).to('cpu').detach().numpy().tolist())
        a.append(tags_val.to('cpu').detach().numpy().tolist())

In [ ]:

clr = classification_report(actual, predicted, target_names=[ 'Others',            # 0
                                                            'College Name',        # 1
                                                            'Degree',              # 2
                                                            'Graduation Year',     # 3
                                                            'Years of Experience', # 4
                                                            'Companies worked at', # 5
                                                            'Designation',         # 6
                                                            'Skills',              # 7
                                                            'Location',            # 8
                                                            'Email Address',       # 9 
                                                            'Name'])              # 10

In [20]:
train_data_dict[200]

KeyError: 200

In [1]:
import numpy as np

In [2]:
def rnn_cell_forward(xt, a_prev, parameters):
    """
    Implements a single forward step of the RNN-cell as described in Figure (2)

    Arguments:
    xt -- your input data at timestep "t", numpy array of shape (n_x, m).
    a_prev -- Hidden state at timestep "t-1", numpy array of shape (n_a, m)
    parameters -- python dictionary containing:
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        ba --  Bias, numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    Returns:
    a_next -- next hidden state, of shape (n_a, m)
    yt_pred -- prediction at timestep "t", numpy array of shape (n_y, m)
    cache -- tuple of values needed for the backward pass, contains (a_next, a_prev, xt, parameters)
    """
    
    # Retrieve parameters from "parameters"
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]
    
    ### START CODE HERE ### (≈2 lines)
    # compute next activation state using the formula given above
    a_next = np.tanh(np.dot(Wax, xt) + np.dot(Waa, a_prev) + ba)
    # compute output of the current cell using the formula given above
    yt_pred = softmax(np.dot(Wya, a_next) + by)
    ### END CODE HERE ###
    
    # store values you need for backward propagation in cache
    cache = (a_next, a_prev, xt, parameters)
    
    return a_next, yt_pred, cache

In [4]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0) 

In [6]:
np.random.seed(1)
xt = np.random.randn(3,10)
a_prev = np.random.randn(5,10)
Waa = np.random.randn(5,5)
Wax = np.random.randn(5,3)
Wya = np.random.randn(2,5)
ba = np.random.randn(5,1)
by = np.random.randn(2,1)
parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "ba": ba, "by": by}

a_next, yt_pred, cache = rnn_cell_forward(xt, a_prev, parameters)
#print("a_next[4] = ", a_next[4])
print("a_next.shape = ", a_next.shape)
#print("yt_pred[1] =", yt_pred[1])
print("yt_pred.shape = ", yt_pred.shape)

a_next.shape =  (5, 10)
yt_pred.shape =  (2, 10)
